# Data Preparation 

**Goal:** Convert raw analytics CSVs into a clean daily dataset at the **video × day** level for modeling.

### What this notebook produces
- `data/processed/daily_features.parquet` — modeling-ready table
- (optional) `data/processed/splits.json` — time split boundaries

### Dataset inputs (local only)
- `data/raw/kaggle_youtube_trending/Aggregated_Metrics_By_Video.csv`
- `data/raw/kaggle_youtube_trending/Video_Performance_Over_Time.csv`

### Granularity choice
We model at **video × day** because the policy later selects a *daily slate* of videos.

## 0) Setup: imports + paths
We keep paths relative to the repo root so the notebook runs on any machine.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

REPO = Path("..")              # because notebook lives in /notebooks
RAW  = REPO / "data" / "raw" / "kaggle_youtube_trending"
OUT  = REPO / "data" / "processed"
OUT.mkdir(parents=True, exist_ok=True)

## 1) Load raw tables + quick sanity checks

We load:
- `Aggregated_Metrics_By_Video`: per-video summary table (small)
- `Video_Performance_Over_Time`: per-day performance table (large)

In [2]:
video = pd.read_csv(RAW / "Aggregated_Metrics_By_Video.csv")
perf  = pd.read_csv(RAW / "Video_Performance_Over_Time.csv")

print("video:", video.shape)
print("perf :", perf.shape)
display(video.head(2))
display(perf.head(2))

video: (224, 19)
perf : (111857, 14)


,Video,Video title,Video pub­lish time,Com­ments ad­ded,Shares,Dis­likes,Likes,Sub­scribers lost,Sub­scribers gained,RPM (USD),CPM (USD),Av­er­age per­cent­age viewed (%),Av­er­age view dur­a­tion,Views,Watch time (hours),Sub­scribers,Your es­tim­ated rev­en­ue (USD),Im­pres­sions,Im­pres­sions click-through rate (%)
0,Total,NaN,NaN,14197,39640,3902,225021,45790,229241,5.276,11.990,26.61,0:03:25,5568487,317602.3536,183451,29068.652,100954064,3.16
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,"May 8, 2020",907,9583,942,46903,451,46904,6.353,12.835,36.65,0:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14


,Date,Video Title,External Video ID,Video Length,Thumbnail link,Views,Video Likes Added,Video Dislikes Added,Video Likes Removed,User Subscriptions Added,User Subscriptions Removed,Average View Percentage,Average Watch Time,User Comments Added
0,19 Jan 2022,Kaggle Project From Scratch - Part 2 (Explorat...,KQ80oD_boBM,2191,https://i.ytimg.com/vi/KQ80oD_boBM/hqdefault.jpg,13,0,0,0,0,0,0.069055,151.300154,0
1,19 Jan 2022,Welcome To My Channel | Ken Jee | Data Science,smeFkHwnM_k,51,https://i.ytimg.com/vi/smeFkHwnM_k/hqdefault.jpg,2,0,0,0,1,0,0.471255,24.034000,0


## 2) Clean column names (fix hidden characters)

Some columns contain soft hyphens or unusual characters (e.g., `pub­lish`).
We normalize columns to consistent snake_case-like labels.

In [4]:
def clean_cols(cols):
    out = []
    for c in cols:
        c = str(c).replace("\xad", "")      # remove soft hyphen
        c = c.strip()
        out.append(c)
    return out

video.columns = clean_cols(video.columns)
perf.columns  = clean_cols(perf.columns)

video.columns.tolist(), perf.columns.tolist()

(['Video',
  'Video title',
  'Video publish time',
  'Comments added',
  'Shares',
  'Dislikes',
  'Likes',
  'Subscribers lost',
  'Subscribers gained',
  'RPM (USD)',
  'CPM (USD)',
  'Average percentage viewed (%)',
  'Average view duration',
  'Views',
  'Watch time (hours)',
  'Subscribers',
  'Your estimated revenue (USD)',
  'Impressions',
  'Impressions click-through rate (%)'],
 ['Date',
  'Video Title',
  'External Video ID',
  'Video Length',
  'Thumbnail link',
  'Views',
  'Video Likes Added',
  'Video Dislikes Added',
  'Video Likes Removed',
  'User Subscriptions Added',
  'User Subscriptions Removed',
  'Average View Percentage',
  'Average Watch Time',
  'User Comments Added'])

## 3) Standardize key columns (rename the fields we will use)

We standardize:
- `External Video ID` → `video_id`
- `Video Title` → `video_title`
- `Date` → `date`

In [7]:
perf = perf.rename(columns={
    "External Video ID": "video_id",
    "Video Title": "video_title",
    "Date": "date",
    "Video Length": "video_length",
    "Views": "views_daily",
    "User Comments Added": "comments_added_daily",
    "Video Likes Added": "likes_added_daily",
    "Video Dislikes Added": "dislikes_added_daily",
    "Video Likes Removed": "likes_removed_daily",
    "User Subscriptions Added": "subs_added_daily",
    "User Subscriptions Removed": "subs_removed_daily",
    "Average View Percentage": "avg_pct_viewed_daily",
    "Average Watch Time": "avg_watch_time_daily",
})


In [9]:
perf["date"] = pd.to_datetime(perf["date"], errors="coerce")
perf[["video_id","video_title","date"]].head()

,video_id,video_title,date
0,KQ80oD_boBM,Kaggle Project From Scratch - Part 2 (Explorat...,2022-01-19
1,smeFkHwnM_k,Welcome To My Channel | Ken Jee | Data Science,2022-01-19
2,vfV4nm004VQ,How She Dominated the FAANG Data Science Inter...,2022-01-19
3,3TrAYmrmA8o,The 9 Books That Changed My Perspective in 2019,2022-01-19
4,Xgg7dIKys9E,Interview with the Director of AI Research @ N...,2022-01-19


## 4) Create next-day targets (supervised learning setup)

We want a daily recommender.  
So for each video on day **t**, we predict how it will perform on day **t+1**.

Targets:
- `y_next_avg_pct_viewed` = next-day average % viewed
- `y_next_watch_time`     = next-day avg watch time (seconds)

This avoids leakage because features come from day t while labels come from day t+1.

In [10]:
perf = perf.sort_values(["video_id","date"])

perf["y_next_avg_pct_viewed"] = perf.groupby("video_id")["avg_pct_viewed_daily"].shift(-1)
perf["y_next_watch_time"]     = perf.groupby("video_id")["avg_watch_time_daily"].shift(-1)

# drop last day per video (no t+1 label)
model_df = perf.dropna(subset=["y_next_avg_pct_viewed","y_next_watch_time"]).copy()

model_df[["y_next_avg_pct_viewed","y_next_watch_time"]].describe()

,y_next_avg_pct_viewed,y_next_watch_time
count,110362.000000,110362.000000
mean,0.350440,216.823668
std,0.232538,190.787976
min,0.000000,0.000000
25%,0.178004,117.403156
50%,0.336828,181.531704
75%,0.476237,268.185524
max,8.476340,5322.300000


## 5) Handle outliers & stabilize heavy tails

`y_next_avg_pct_viewed` should behave like a fraction (roughly 0–1), but extreme values exist.
We apply a conservative cap (winsorization) to reduce distortion.

For watch time, the distribution is heavy-tailed, so we model:
- `y_next_watch_time_log = log1p(y_next_watch_time)`

In [12]:
# cap percent viewed
cap = model_df["y_next_avg_pct_viewed"].quantile(0.995)
model_df["y_next_avg_pct_viewed"] = model_df["y_next_avg_pct_viewed"].clip(lower=0, upper=cap)

# log transform watch time target
model_df["y_next_watch_time_log"] = np.log1p(model_df["y_next_watch_time"])

model_df[["y_next_avg_pct_viewed","y_next_watch_time","y_next_watch_time_log"]].describe()

,y_next_avg_pct_viewed,y_next_watch_time,y_next_watch_time_log
count,110362.000000,110362.000000,110362.000000
mean,0.349335,216.823668,5.025432
std,0.225807,190.787976,1.016815
min,0.000000,0.000000,0.000000
25%,0.178004,117.403156,4.774095
50%,0.336828,181.531704,5.206924
75%,0.476237,268.185524,5.595401
max,1.043220,5322.300000,8.579849


## 6) Feature set (baseline)

We start with simple behavioral signals available at day t:
- views, likes, dislikes, comments, subscriptions, video_length

We intentionally avoid complex text features in the baseline.

In [14]:
feat_cols = [
    "views_daily","likes_added_daily","dislikes_added_daily","likes_removed_daily",
    "comments_added_daily","subs_added_daily","subs_removed_daily",
    "video_length"
]

keep_cols = ["video_id","video_title","date"] + feat_cols + ["y_next_avg_pct_viewed","y_next_watch_time_log"]
final_df = model_df[keep_cols].copy()

final_df.head()

,video_id,video_title,date,views_daily,likes_added_daily,dislikes_added_daily,likes_removed_daily,comments_added_daily,subs_added_daily,subs_removed_daily,video_length,y_next_avg_pct_viewed,y_next_watch_time_log
91434,-3d1NctSv0c,"Ken Jee Q & A Live Stream (50,000 Sub Special!)",2020-06-18,0,10,0,0,0,2,0,3413,0.089764,5.728037
91405,-3d1NctSv0c,"Ken Jee Q & A Live Stream (50,000 Sub Special!)",2020-06-19,1378,89,1,2,0,2,4,3413,0.110562,5.935818
91242,-3d1NctSv0c,"Ken Jee Q & A Live Stream (50,000 Sub Special!)",2020-06-20,330,14,0,0,0,2,4,3413,0.127342,6.076769
91197,-3d1NctSv0c,"Ken Jee Q & A Live Stream (50,000 Sub Special!)",2020-06-21,114,4,0,0,0,1,4,3413,0.108499,5.917026
91002,-3d1NctSv0c,"Ken Jee Q & A Live Stream (50,000 Sub Special!)",2020-06-22,69,3,0,0,0,1,0,3413,0.260563,6.791561


## 7) Time-based split (train/valid/test)

We split by date to mimic production:
- train: earliest period
- valid: middle period (for model selection)
- test : latest period (final evaluation)

This prevents leakage across time and matches real deployment conditions.

In [15]:
#time-based spilt
model_df["date"] = pd.to_datetime(model_df["date"])
model_df = model_df.sort_values(["date", "video_id"]).reset_index(drop=True)

d1 = model_df["date"].quantile(0.60)
d2 = model_df["date"].quantile(0.80)

train_df = model_df[model_df["date"] <= d1].copy()
valid_df = model_df[(model_df["date"] > d1) & (model_df["date"] <= d2)].copy()
test_df  = model_df[model_df["date"] > d2].copy()

print(train_df.shape, valid_df.shape, test_df.shape)
print(train_df["date"].min(), train_df["date"].max())
print(valid_df["date"].min(), valid_df["date"].max())

(60382, 17) (20240, 17) (19954, 17)
2017-11-01 00:00:00 2021-04-28 00:00:00
2021-04-29 00:00:00 2021-08-27 00:00:00


## 8) Save processed datasets

These files are the interface between Notebook 1 and Notebook 2.
Notebook 2 should start by loading these instead of re-cleaning everything.

In [16]:
train_df.to_parquet(OUT / "train_daily.parquet", index=False)
valid_df.to_parquet(OUT / "valid_daily.parquet", index=False)
test_df.to_parquet(OUT / "test_daily.parquet", index=False)

print("Saved:", OUT)

Saved: ../data/processed
